In [0]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn import datasets, model_selection

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd 'gdrive/My Drive'

/content/gdrive/My Drive


In [5]:
!ls

 3장		    faces_images	  simple_model.h5     trec
'Colab Notebooks'   __MACOSX		  simple_weights.h5
 data		    output.csv		  somedummyfile
 dataset	    pytlesson_notebooks   test_vision.csv


In [0]:
data_filenames = pd.read_csv('dataset/train_vision.csv')

In [7]:
data_filenames['filename']

0       face_3267.png
1        face_587.png
2       face_6585.png
3       face_3094.png
4       face_1170.png
            ...      
5845     face_692.png
5846    face_5357.png
5847    face_3280.png
5848    face_6319.png
5849    face_2912.png
Name: filename, Length: 5850, dtype: object

In [8]:
data_filenames['label'] - 1

0       0
1       3
2       3
3       3
4       3
       ..
5845    3
5846    3
5847    0
5848    3
5849    0
Name: label, Length: 5850, dtype: int64

In [0]:
data = [] # 설명변수
label = [] # 목적변수

for lbl, filename in zip(data_filenames['label'] - 1, data_filenames['filename']):
  img = Image.open('faces_images/' + filename, 'r')
  r, g, b = img.split()
  # 채널별로 분리해서 [0,1] 구간으로 정규화
  r_resize_img = np.asarray(np.float32(r)/255.0)
  g_resize_img = np.asarray(np.float32(g)/255.0)
  b_resize_img = np.asarray(np.float32(b)/255.0)
  rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
  # 가공된 이미지를 리스트에 추가
  data.append(rgb_resize_img)
  
  # 이미지 레이블을 리스트에 추가
  label.append(lbl)

In [10]:
# NumPy 배열로 변환
data = np.array(data, dtype='float32')
label = np.array(label, dtype='int64')
print(data.shape)

(5850, 3, 128, 128)


In [11]:
# 데이터 집합을 훈련 데이터와 테스트 데이터로 분할
train_X, validation_X, train_Y, validation_Y = model_selection.train_test_split(
    data, label)

# 데이터 건수를 확인
print(len(train_X))
print(len(validation_X))

4387
1463


In [12]:
# 훈련 데이터 텐서 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

# 텐서 크기를 화면에 출력
print(train_X.shape)
print(train_Y.shape)

torch.Size([4387, 3, 128, 128])
torch.Size([4387])


In [13]:
# 설명변수와 목적변수 텐서를 합치기
train = TensorDataset(train_X, train_Y)

# 첫 번째 텐서의 내용을 화면에 출력
print(train[0])
# 미니배치로 분할하기
train_loader = DataLoader(train, batch_size=32, shuffle=True)

(tensor([[[0.3255, 0.2902, 0.2824,  ..., 0.2431, 0.2471, 0.2588],
         [0.3059, 0.2941, 0.2824,  ..., 0.2471, 0.2431, 0.2549],
         [0.2941, 0.2902, 0.2863,  ..., 0.2510, 0.2471, 0.2510],
         ...,
         [0.2980, 0.2941, 0.2980,  ..., 0.2902, 0.2902, 0.2667],
         [0.3020, 0.3020, 0.2980,  ..., 0.2392, 0.2824, 0.2824],
         [0.3176, 0.3098, 0.3059,  ..., 0.1608, 0.1647, 0.1922]],

        [[0.2431, 0.2196, 0.2118,  ..., 0.1922, 0.1922, 0.2000],
         [0.2353, 0.2275, 0.2196,  ..., 0.1961, 0.1961, 0.1961],
         [0.2235, 0.2235, 0.2196,  ..., 0.2000, 0.1961, 0.1961],
         ...,
         [0.2431, 0.2353, 0.2353,  ..., 0.2275, 0.2275, 0.2196],
         [0.2431, 0.2392, 0.2392,  ..., 0.1804, 0.2157, 0.2235],
         [0.2549, 0.2510, 0.2510,  ..., 0.1255, 0.1373, 0.1569]],

        [[0.4510, 0.4314, 0.4314,  ..., 0.3922, 0.3922, 0.4000],
         [0.4392, 0.4275, 0.4353,  ..., 0.3922, 0.3922, 0.3882],
         [0.4392, 0.4392, 0.4314,  ..., 0.3922, 0.3922, 0

In [0]:
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 합성곱층
        self.conv1 = nn.Conv2d(3, 10, 5) # 입력 채널 수, 출력 채널 수, 필터 크기
        self.conv2 = nn.Conv2d(10, 20, 5)

        # 전결합층
        self.fc1 = nn.Linear(20 * 29 * 29, 64) # 29=(((((128-5)+1)/2)-5)+1)/2
        self.fc2 = nn.Linear(64, 6)
    
    def forward(self, x):
        # 풀링층
        x = F.max_pool2d(F.relu(self.conv1(x)), 2) # 풀링 영역 크기
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 20 * 29 * 29) # 다른 차원으로부터 해당 값을 유추
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
# 인스턴스 생성
model = Net()

In [17]:
# 오차함수 객체
criterion = nn.CrossEntropyLoss()

# 최적화를 담당할 객체
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 시작
for epoch in range(100):
    total_loss = 0
    acc = 0
    correct = 0
    # 분할해 둔 데이터를 꺼내옴
    for train_x, train_y in train_loader:
        # 계산 그래프 구성
        train_x, train_y = Variable(train_x), Variable(train_y)
        # 경사 초기화
        optimizer.zero_grad()
        # 순전파 계산
        output = model(train_x)
        # 오차 계산
        loss = criterion(output, train_y)
        # 역전파 계산
        loss.backward()
        # 가중치 업데이트
        optimizer.step()
        # 누적 오차 계산
        total_loss += loss.data
    print("epoch: %d, loss: %.6f" % (epoch+1,loss.data))

print('Finished Training')

epoch: 1, loss: 1.695400
epoch: 2, loss: 0.162587
epoch: 3, loss: 0.370493
epoch: 4, loss: 0.089663
epoch: 5, loss: 0.566741
epoch: 6, loss: 0.229720
epoch: 7, loss: 0.314507
epoch: 8, loss: 0.081805
epoch: 9, loss: 0.071086
epoch: 10, loss: 0.356867
epoch: 11, loss: 0.446174
epoch: 12, loss: 0.221624
epoch: 13, loss: 0.609326
epoch: 14, loss: 0.016373
epoch: 15, loss: 0.075536
epoch: 16, loss: 0.068914
epoch: 17, loss: 0.000000
epoch: 18, loss: 0.010495
epoch: 19, loss: 0.048288
epoch: 20, loss: 0.024637
epoch: 21, loss: 0.000099
epoch: 22, loss: 0.104535
epoch: 23, loss: 0.000613
epoch: 24, loss: 0.000835
epoch: 25, loss: 0.020190
epoch: 26, loss: 0.118072
epoch: 27, loss: 0.150974
epoch: 28, loss: 0.000002
epoch: 29, loss: 0.000327
epoch: 30, loss: 0.016115
epoch: 31, loss: 0.012976
epoch: 32, loss: 0.238380
epoch: 33, loss: 0.022735
epoch: 34, loss: 0.000248
epoch: 35, loss: 0.004654
epoch: 36, loss: 0.179551
epoch: 37, loss: 0.000014
epoch: 38, loss: 0.000001
epoch: 39, loss: 0.00

In [18]:
# NumPy 배열로 변환
validation_X = np.array(validation_X, dtype='float32')
validation_Y = np.array(validation_Y, dtype='int64')

# 테스트 데이터를 텐서로 변환
validation_X = torch.from_numpy(validation_X).float()
validation_Y = torch.from_numpy(validation_Y).long()

# 텐서 크기 확인
print(validation_X.shape)
print(validation_Y.shape)

torch.Size([1463, 3, 128, 128])
torch.Size([1463])


In [19]:
# 계산 그래프 구성
validation_x, validation_y = Variable(validation_X), Variable(validation_Y)
# 출력이 0 혹은 1이 되게 함
result = torch.max(model(validation_x).data, 1)[1]
# 모형의 정확도 측정
accuracy = sum(validation_y.data.numpy() == result.numpy()) / len(validation_y.data.numpy())

# 모형의 정확도 출력
accuracy

0.861244019138756

In [0]:
test_filenames = pd.read_csv('dataset/test_vision.csv')

In [21]:
test_filenames['filename']

0       face_2624.png
1       face_3385.png
2       face_5015.png
3       face_6715.png
4       face_6171.png
            ...      
1995     face_489.png
1996    face_7601.png
1997    face_2020.png
1998    face_4219.png
1999     face_269.png
Name: filename, Length: 2000, dtype: object

In [0]:
test_data = [] # 설명변수

for test_filename in test_filenames['filename']:
  img = Image.open('faces_images/' + test_filename, 'r')
  r, g, b = img.split()
  # 채널별로 분리해서 [0,1] 구간으로 정규화
  r_resize_img = np.asarray(np.float32(r)/255.0)
  g_resize_img = np.asarray(np.float32(g)/255.0)
  b_resize_img = np.asarray(np.float32(b)/255.0)
  rgb_resize_img = np.asarray([r_resize_img, g_resize_img, b_resize_img])
  # 가공된 이미지를 리스트에 추가
  test_data.append(rgb_resize_img)

In [23]:
# NumPy 배열로 변환
test_data = np.array(test_data, dtype='float32')

# 훈련 데이터 텐서 변환
test_data = torch.from_numpy(test_data).float()

# 텐서 크기를 화면에 출력
print(test_data.shape)

torch.Size([2000, 3, 128, 128])


In [0]:
# 미니배치로 분할하기
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=1)

In [0]:
import csv
from operator import itemgetter
model.eval()
 
data_list = []
f = open('output.csv','w',newline='')
csvWriter = csv.DictWriter(f,['prediction'])
csvWriter.writerows([{'prediction':'prediction'}])

for data in test_loader:
    test_data = data

    outputs = model(test_data)
    
    prediction_tensor = torch.max(outputs.data, 1)[1]

    prediction = prediction_tensor.numpy()
    
    data_list.append({'prediction': prediction.data[0] + 1})
 
csvWriter.writerows(data_list)
f.close()